In [3]:
#imports
import pandas as pd 
import numpy as np
import pickle
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction import text
import datetime
import ast

In [4]:
data = pd.read_csv('../nowdata/parsing/combined_df.csv', sep = "\t", low_memory=False, encoding="utf-8")

In [5]:
data['WEBTEXT'] = data['WEBTEXT'].apply(ast.literal_eval)

In [6]:
data.loc[0,"WEBTEXT"][2][3]

'\nHFA: Elementary School is located in the recently transformed Duane Doty Elementary School in Detroit’s Northend community.\xa0\nThis historic building is on the edge of the Boston-Edison District, just west of Woodward, between the Davison Freeway and Chicago Blvd.\xa0\n10225 Third Street, Detroit MI 48202\n313.826.1159 | \nhfaes@hfli.org\nHFA: Elementary School Location\n10225 3rd St\nSign in\n|\nReport Abuse\n|\nPowered By \nGoogle Sites'

In [7]:
data['WEBTEXT'] = data['WEBTEXT'].fillna('0')

In [8]:
#webtext = data["WEBTEXT"]
#webtext_with_text = []
#for t in webtext:
#    try:
#        len_site = len(t)
#        for i in range(len_site):
#            webtext_with_text.append(t[i][3])
#    except:
#        pass

webtext = data["WEBTEXT"]
webtext_with_text = []
tokens = []
for t in webtext:
    try:
        len_site = len(t)
        #initializing a master string
        school_string = ''
        for i in range(len_site):
            school_string += t[i][3] + ' \n '
        webtext_with_text.append(school_string)
    except:
        pass

In [9]:
no_features = 1000
# creating custom stop words
my_additional_stop_word_list = []
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
my_additional_stop_word_list.append('00')
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, max_features=no_features, stop_words=stop_words_extra)
tf = tf_vectorizer.fit_transform(webtext_with_text)
tf_feature_names = tf_vectorizer.get_feature_names()

In [10]:
# No. of topics - 10
no_topics = 20
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [11]:
#old version
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic " + str(topic_idx) + ': ' + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)
#topics to note: topic 1, topic 2, topic 4, topic , topic 15

Topic 0: charter new org children public family academy teachers parents like child news teacher parent board day blog style info support
Topic 1: life skills board elementary arizona children learn work time help support read child meeting news phoenix az teachers make north
Topic 2: college academy university course state online science support county services curriculum courses teachers academic technology work social help time board
Topic 3: email phone calendar send fax powered academy pm edlio directions events homework charter parent ca facebook time copy form portal
Topic 4: district montessori elementary pm website fax phone rights reserved board wi email 06 page home 01 09 river http teacher
Topic 5: harmony value la public title act en website ix section policy accessibility programs el children access site los las regarding
Topic 6: site map houston close blackboard web success tx stem privacy policy raul questions link copyright use rights terms broadway reserved
Topic 7: 

In [21]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.046188  0.142840       1        1  10.109472
17     0.109403  0.004936       2        1   7.058178
2     -0.017975  0.020519       3        1   7.049711
18    -0.124152  0.128722       4        1   6.835850
9      0.081838  0.003407       5        1   6.583529
13    -0.005937 -0.046976       6        1   6.277003
0      0.037034  0.032312       7        1   5.512957
6     -0.021785 -0.019560       8        1   5.393036
7      0.038418  0.034804       9        1   5.036563
1      0.027982  0.034704      10        1   5.017061
8      0.059083 -0.012903      11        1   4.941702
11     0.007830 -0.059634      12        1   4.889842
19    -0.007416 -0.040443      13        1   4.537110
4      0.035240 -0.073420      14        1   4.171334
3      0.123136 -0.048833      15        1   4.088695
15    -0.061502 -0.096906      16        1   3.141083
10    -0.050254  0.007050      17        1   2.982110
16    -0.031636 -0.020583      18        1   2.388185
5     -0.182453 -0.072229      19        1   2.196545
12    -0.063042  0.082194      20        1   1.790035, topic_info=     Category           Freq        Term          Total  loglift  logprob
term                                                                     
453   Default  272950.000000        idea  272950.000000  30.0000  30.0000
786   Default  121602.000000         san  121602.000000  29.0000  29.0000
27    Default  507052.000000     academy  507052.000000  28.0000  28.0000
161   Default  410750.000000     charter  410750.000000  27.0000  27.0000
499   Default  112913.000000          la  112913.000000  26.0000  26.0000
399   Default  416504.000000       grade  416504.000000  25.0000  25.0000
869   Default  245917.000000     success  245917.000000  24.0000  24.0000
296   Default  271169.000000       email  271169.000000  23.0000  23.0000
185   Default  317029.000000     college  317029.000000  22.0000  22.0000
899   Default   99369.000000       texas   99369.000000  21.0000  21.0000
273   Default  193392.000000    district  193392.000000  20.0000  20.0000
545   Default   66232.000000         los   66232.000000  19.0000  19.0000
301   Default   67076.000000          en   67076.000000  18.0000  18.0000
728   Default  279216.000000      public  279216.000000  17.0000  17.0000
26    Default   87911.000000   academies   87911.000000  16.0000  16.0000
292   Default   71553.000000          el   71553.000000  15.0000  15.0000
71    Default   58046.000000     antonio   58046.000000  14.0000  14.0000
821   Default  144024.000000        site  144024.000000  13.0000  13.0000
664   Default  219857.000000       phone  219857.000000  12.0000  12.0000
126   Default  335903.000000       board  335903.000000  11.0000  11.0000
447   Default   68647.000000     houston   68647.000000  10.0000  10.0000
806   Default   73222.000000        send   73222.000000   9.0000   9.0000
143   Default  239083.000000    calendar  239083.000000   8.0000   8.0000
123   Default   43954.000000  blackboard   43954.000000   7.0000   7.0000
99    Default   63658.000000      austin   63658.000000   6.0000   6.0000
674   Default  140616.000000          pm  140616.000000   5.0000   5.0000
560   Default   83126.000000         map   83126.000000   4.0000   4.0000
946   Default   48841.000000      valley   48841.000000   3.0000   3.0000
947   Default   42140.000000       value   42140.000000   2.0000   2.0000
832   Default   74849.000000       south   74849.000000   1.0000   1.0000
...       ...            ...         ...            ...      ...      ...
483   Topic20    3222.713502          ix   35482.762483   1.6241  -5.5860
308   Topic20    1952.625214  enrichment   24329.092366   1.5004  -6.0871
614   Topic20    2047.992758   nutrition   25730.901880   1.4921  -6.0394
309   Topic20    6772.091063      enroll   88312.045863   1.4549  -4.8434
871   Topic20    3787.314539   

## NER

In [19]:
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load('en')

ModuleNotFoundError: No module named 'spacy'

In [ ]:
#regex code to remove unicode/ \n's, etc
re.sub(r'\\n', ' ', text_str)
re.sub(r'\\xa', ' ', text_str)


In [ ]:
#using st.tag to create list with tupled categorization for entities
tag_words = st.tag(master_string.split())
#filtering out the non named entities
named_e = [t for t in tag_lyrics if t[1] != 'O']
named_e

## Testing

### NMF

In [ ]:
with open('../nowdata/traincf_2015_15_250_counts.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
webtext = data["WEBTEXT"]
webtext_nmf = []
tokens = []
for t in webtext:
    try:
        len_site = len(t)
        #initializing a list
        school_lst = []
        for i in range(len_site):
            school_lst.append(t[i][3]) 
        webtext_nmf.append(school_lst)
    except:
        pass

In [ ]:
no_features = 1000
# creating custom stop words
my_additional_stop_word_list = []
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
my_additional_stop_word_list.append('00')
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words=stop_words_extra)
tfidf = tfidf_vectorizer.fit_transform(webtext_nmf)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

In [ ]:
def display_topics_nmf(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print documents[doc_index]

In [ ]:
no_top_words = 20
no_top_documents = 20
display_topics_nmf(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)